In [1]:
import polars as pl
import matplotlib.pyplot as plt
import sys
import ase
import json
import numpy as np
import scipy.optimize
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from pathlib import Path
from enum import Enum
from ase.units import Hartree, kcal, mol
from dftmp2bench.pareto import is_pareto_efficient
from scipy.stats import linregress

project_path = Path("/cluster/home/schmidtn/master/wahlprak/dftmp2bench")
output_path = Path(
    "/cluster/home/schmidtn/master/wahlprak/dft-mp2-data/ala_dih_cons_opt_test/orca/ALA_neutral_singlet.xyz/rimp2/def2svp/ALA_neutral_singlet.xyz.fc73ddcf-e3d8-11ef-8425-5379090b8daf.property.json"
)
# output_path = Path("/cluster/home/schmidtn/master/wahlprak/dft-mp2-data/benchmark/orca/ala0.xyz/hf/631+gd/ala0.xyz.0022dbf8-d9a2-11ef-abea-d85ed34e2189.property.json")

In [2]:
# pl.read_json(output_path, schema=["GRAD"])
output_path_ = Path(
    "/cluster/home/schmidtn/master/wahlprak/"
    "dft-mp2-data/ala_dih_cons_opt_test/orca/ALA_neutral_singlet.xyz/"
    "rimp2/def2svp/ALA_neutral_singlet.xyz.fc73ddcf-e3d8-11ef-8425-5379090b8daf.out"
)

import cclib
import cclib.io

# cclib.io.ccread(output_path_)

import ase
import ase.io

ase.io.read(output_path)

JSONDecodeError: Expecting ',' delimiter: line 71 column 32 (char 3570)

In [ ]:
with open(output_path) as f:
    data_dict = json.load(f)
print(data_dict["Calculation_Status"]["STATUS"])


# import ast

# with open(output_path) as f:
#     data_string = f.read()
# #print(data_string)
# data_dict = ast.literal_eval(data_string)

In [ ]:
output_path

In [ ]:
"GRAD"

In [3]:
def recover_gradients_from_malformed_json(output_path, natoms=42):
    """Unofficial implementation of the 'find out' function
    ('fuck around' function not yet public)"""

    with open(output_path) as f:
        output_str = f.read()

    no_ws = output_str.replace(" ", "")
    no_ws = no_ws.replace("\n", "")
    no_ws_reversed = no_ws[::-1]

    if not "DARG" in no_ws_reversed:
        # return np.zeros(natoms, 3)
        return None

    darg_idx = no_ws_reversed.index("DARG")
    darg_hit_prefix = no_ws_reversed[0:darg_idx:1][::-1]
    close_bracket_idx = darg_hit_prefix.index("}")

    txt_array = darg_hit_prefix[2:close_bracket_idx]
    txt_array = txt_array.replace("[", " ")
    txt_array = txt_array.replace("]", " ")

    from io import StringIO

    s = StringIO(txt_array)

    data = np.genfromtxt(s, delimiter=",")
    forces = data.reshape(natoms, 3)
    return forces

In [4]:
recover_gradients_from_malformed_json(output_path)

array([[-5.98761648e-06, -1.43875318e-05, -2.61569054e-06],
       [-2.49356193e-06, -2.05156966e-06, -5.84452599e-07],
       [ 3.41659687e-06, -6.11957793e-06, -6.01215766e-06],
       [-3.34251532e-05,  2.85339790e-05, -1.33189480e-06],
       [ 1.27844383e-05,  1.22971631e-05,  3.14121105e-05],
       [ 5.40002840e-06,  2.01029216e-06, -1.23922745e-06],
       [-1.99579993e-05, -8.55981662e-07, -1.41616849e-06],
       [ 1.02492281e-04,  9.53314196e-04, -1.40141715e-03],
       [-3.84466314e-06,  4.09659698e-06,  1.81236051e-06],
       [ 5.73494239e-06, -9.11789500e-06, -1.07459323e-06],
       [ 1.60775128e-06,  2.37557827e-06, -5.36020336e-06],
       [ 2.79358730e-05,  4.83999456e-05,  2.35901282e-05],
       [-2.44624989e-04, -9.89495817e-04,  7.83562382e-04],
       [-2.23937798e-05,  5.24943079e-05, -2.87575696e-05],
       [ 1.64130737e-04, -3.91482203e-04,  1.58873655e-03],
       [ 9.24575504e-06,  1.05009577e-05, -6.13235823e-07],
       [ 4.62242170e-06,  1.75508831e-06

In [5]:
top_path = Path(
    "/cluster/home/schmidtn/master/wahlprak/dft-mp2-data/ala_dih_cons_opt_test/orca/ALA_neutral_singlet.xyz/"
)
list_of_jsons = list(top_path.glob("*/*/*json"))
gradients = [recover_gradients_from_malformed_json(_) for _ in list_of_jsons]
# gradients

In [6]:
def rmsd(a, b):
    return (np.mean((a - b) ** 2)) ** 0.5

In [7]:
json_grad_dict = {
    str(k): v.flatten() for k, v in zip(list_of_jsons, gradients) if v is not None
}
pl.DataFrame(json_grad_dict).write_csv("gradients.csv")

data_dict = {
    "key1": [],
    "key2": [],
    "rmsd": [],
    "ln_rmsd": [],
    "functional1": [],
    "basis1": [],
    "functional2": [],
    "basis2": [],
}

done_key_pairs = []


for k in json_grad_dict.keys():
    for k2 in json_grad_dict.keys():
        if k != k2 and (k2, k) not in done_key_pairs:
            data_dict["key1"].append(k)
            data_dict["key2"].append(k2)
            data_dict["ln_rmsd"].append(
                np.log(2240.874995 * rmsd(json_grad_dict[k], json_grad_dict[k2]))
            )
            data_dict["rmsd"].append(
                2240.874995 * rmsd(json_grad_dict[k], json_grad_dict[k2])
            )

            path_split1 = str(k).split("/")
            path_split2 = str(k2).split("/")
            data_dict["functional1"].append(path_split1[-3])
            data_dict["functional2"].append(path_split2[-3])
            data_dict["basis1"].append(path_split1[-2])
            data_dict["basis2"].append(path_split2[-2])
            done_key_pairs.append((k, k2))

/tmp/ipykernel_1058455/1773319760.py:26: RuntimeWarning: divide by zero encountered in log
  np.log(2240.874995 * rmsd(json_grad_dict[k], json_grad_dict[k2]))


In [23]:
df = pl.DataFrame(data_dict)

df["key1"][-3], df["key2"][-3]
print(df("functional1").unique())

TypeError: 'DataFrame' object is not callable

In [9]:
just_rimp2_comparisons = df.filter(
    (pl.col("functional1") == "rimp2") | (pl.col("functional2") == "rimp2")
)

In [10]:
just_rimp2_comparisons.sort("rmsd")

key1,key2,rmsd,ln_rmsd,functional1,basis1,functional2,basis2
str,str,f64,f64,str,str,str,str
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.275858,-1.287868,"""rimp2""","""def2svp""","""pbe0""","""def2svp"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.347801,-1.056126,"""hf""","""def2svp""","""rimp2""","""def2svp"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.391529,-0.937697,"""hf""","""631+gd""","""rimp2""","""def2svp"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.5345,-0.626424,"""rimp2""","""def2svp""","""pbe0""","""631+gd"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.622597,-0.473855,"""rimp2""","""def2svp""","""b2plyp""","""def2svp"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.626725,-0.467247,"""rimp2""","""def2svp""","""PBEh-3c""","""def2svp"""
"""/cluster/home/schmidtn/master/…","""/cluster/home/schmidtn/master/…",0.626725,-0.467247,"""rimp2""","""def2svp""","""PBEh-3c""","""631+gd"""


In [17]:
table = just_rimp2_comparisons.select(["functional1", "basis1", "functional2", "basis2", "rmsd", "ln_rmsd"])
table.to_pandas().to_latex()


'\\begin{tabular}{lllllrr}\n\\toprule\n & functional1 & basis1 & functional2 & basis2 & rmsd & ln_rmsd \\\\\n\\midrule\n0 & hf & def2svp & rimp2 & def2svp & 0.347801 & -1.056126 \\\\\n1 & hf & 631+gd & rimp2 & def2svp & 0.391529 & -0.937697 \\\\\n2 & rimp2 & def2svp & pbe0 & def2svp & 0.275858 & -1.287868 \\\\\n3 & rimp2 & def2svp & pbe0 & 631+gd & 0.534500 & -0.626424 \\\\\n4 & rimp2 & def2svp & PBEh-3c & def2svp & 0.626725 & -0.467247 \\\\\n5 & rimp2 & def2svp & PBEh-3c & 631+gd & 0.626725 & -0.467247 \\\\\n6 & rimp2 & def2svp & b2plyp & def2svp & 0.622597 & -0.473855 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [11]:
40 * 3600 / 50 / 60 / 24

2.0

In [12]:
7.5 * 40 * 3600 / (8 * 50) / 60 / 24

1.875